In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np
import pandas as pd
from build_features import build_data_sets
from build_model import check_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('../data/processed/otodom_cleaned.csv', index_col=0)

X_train, X_test, y_train, y_test = build_data_sets(df=df)

In [4]:
df.describe()

,price,price_m2,area_m2,rooms,floor,parking,build_yr,elevator,max_floor,balcony,backyard,terrace,building_age
count,1.753000e+03,1753.000000,1753.000000,1751.000000,1737.000000,1753.000000,1517.000000,1753.000000,1704.000000,1753.000000,1753.000000,1753.000000,1517.000000
mean,8.436831e+05,12696.556189,64.435220,2.816105,2.377950,0.524815,1996.787739,0.438677,5.071009,0.544210,0.115231,0.146606,25.212261
std,6.808962e+05,5613.125490,30.839518,0.938931,2.231378,0.499526,28.309187,0.496367,4.232692,0.498184,0.319392,0.353813,28.309187
min,4.800000e+04,3200.000000,15.000000,1.000000,-0.500000,0.000000,1920.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-2.000000
25%,4.690000e+05,9259.000000,45.600000,2.000000,1.000000,0.000000,1976.000000,0.000000,3.000000,0.000000,0.000000,0.000000,1.000000
50%,6.200000e+05,11200.000000,58.790000,3.000000,2.000000,1.000000,2009.000000,0.000000,4.000000,1.000000,0.000000,0.000000,13.000000
75%,9.500000e+05,14300.000000,75.610000,3.000000,3.000000,1.000000,2021.000000,1.000000,5.000000,1.000000,0.000000,0.000000,46.000000
max,5.999000e+06,82569.000000,500.000000,7.000000,10.000000,1.000000,2024.000000,1.000000,34.000000,1.000000,1.000000,1.000000,102.000000


In [5]:
gb_model = GradientBoostingRegressor(random_state=42)

params = [
    {
        "loss": ["squared_error", "absolute_error"],
        "learning_rate": [0.01, 0.1, 1],
        "n_estimators": [100,150],
        "min_samples_split": [4,6],
        "max_depth": [3,5,7]
    }
]

grid_result = GridSearchCV(gb_model, params, cv=10, verbose=1, scoring="neg_mean_absolute_error")
grid_result.fit(X_train, y_train)

Fitting 10 folds for each of 72 candidates, totalling 720 fits


GridSearchCV(cv=10, estimator=GradientBoostingRegressor(random_state=42),
             param_grid=[{'learning_rate': [0.01, 0.1, 1],
                          'loss': ['squared_error', 'absolute_error'],
                          'max_depth': [3, 5, 7], 'min_samples_split': [4, 6],
                          'n_estimators': [100, 150]}],
             scoring='neg_mean_absolute_error', verbose=1)

In [6]:
print(grid_result.best_estimator_)

GradientBoostingRegressor(loss='absolute_error', max_depth=7,
                          min_samples_split=4, n_estimators=150,
                          random_state=42)


In [7]:
model1 = GradientBoostingRegressor(loss='absolute_error', max_depth=9, min_samples_split=6, n_estimators=150, random_state=42)
check_model(model1, X_train, X_test, y_train, y_test)

	 train	|  test
mae	 51176 |  143253
r2	 0.95 	|  0.72
rmse	 151095 |  329343


In [8]:
model2 = GradientBoostingRegressor(loss='absolute_error', max_depth=4, min_samples_split=7, n_estimators=150, random_state=42)
check_model(model2, X_train, X_test, y_train, y_test)

	 train	|  test
mae	 123664 |  165291
r2	 0.8 	|  0.63
rmse	 308041 |  377374


# Model evaluation

In [9]:
pred = model2.predict(X_test)

eval_dct = {
    "true": y_test,
    'pred': np.round(pred,1)
}

eval_df = pd.DataFrame(eval_dct)

eval_df['abs_err'] = np.abs(eval_df.true - eval_df.pred)

eval_df.describe()

,true,pred,abs_err
count,3.510000e+02,3.510000e+02,3.510000e+02
mean,8.402655e+05,8.050972e+05,1.652914e+05
std,6.203655e+05,4.653425e+05,3.397336e+05
min,2.600000e+05,3.126540e+05,4.300000e+01
25%,4.790000e+05,4.926999e+05,2.021195e+04
50%,6.200000e+05,6.392830e+05,6.017510e+04
75%,9.650000e+05,9.963693e+05,1.531065e+05
max,5.400000e+06,3.249827e+06,3.821088e+06


in my dataset 75% of all apartments had price below 1_000_000pln.

In [10]:
low_price = eval_df.loc[eval_df.true < 1_000_000]
high_price = eval_df.loc[eval_df.true >= 1_000_000]

In [11]:
low_price.describe()

,true,pred,abs_err
count,272.000000,2.720000e+02,272.000000
mean,594375.088235,6.238953e+05,81784.688603
std,180406.570007,2.301264e+05,112877.405369
min,260000.000000,3.126540e+05,43.000000
25%,453483.750000,4.734680e+05,15161.975000
50%,556670.500000,5.536172e+05,49694.200000
75%,721500.000000,7.254756e+05,91008.975000
max,999350.000000,1.547788e+06,681701.900000


In [12]:
high_price.describe()

,true,pred,abs_err
count,7.900000e+01,7.900000e+01,7.900000e+01
mean,1.686876e+06,1.428982e+06,4.528082e+05
std,8.228290e+05,5.281604e+05,6.046441e+05
min,1.000000e+06,7.666420e+05,6.114600e+03
25%,1.150000e+06,1.099036e+06,1.248427e+05
50%,1.361713e+06,1.306700e+06,2.289462e+05
75%,1.925440e+06,1.530768e+06,5.505611e+05
max,5.400000e+06,3.249827e+06,3.821088e+06
